In [12]:
import warnings
warnings.filterwarnings('ignore')

In [179]:
import pandas as pd
import numpy as np
import random
from keras.layers import Input, Embedding, Dot, Reshape, Dense, BatchNormalization, Dropout
from keras.models import Model

In [14]:
df = pd.read_csv('train.csv')

In [15]:
user_item = pd.DataFrame(columns=['UserId','ItemId'])

In [16]:
itemId_max=[]
for i,row in df.iterrows():
        user_item.loc[i,'UserId'] = int(row[0])
        user_item.loc[i,'ItemId'] = [int(x) for x in row[1].split()]
        itemId_max.append(max(user_item.loc[i,'ItemId']))

In [17]:
num_users = max(user_item['UserId']) + 1
num_items = max(itemId_max) + 1

In [18]:
print(num_users,num_items)

4454 3260


In [19]:
'''
sample positive pairs (uid, item_pos)in a list
'''
def create_pairs(user_item, uid):
    pairs = []
    def u2i(pairs, x):
        for index in x["ItemId"]:
            pairs.append((uid[x["UserId"]], index))
    user_item.apply(lambda x: u2i(pairs, x), axis=1) 
    return pairs

In [20]:
pairs = create_pairs(user_item, user_item['UserId'])

In [177]:
def embedding_model(d):
    user = Input(name = 'user', shape = [None,])    
    user_embedding = Embedding(name = 'user_embedding',
                               input_dim = num_users,
                               output_dim = d, input_length = 1, 
                               embedding_constraint=non_neg())(user)
    Dropout(0.2)
    user_embedding = BatchNormalizetion()(user_embedding)
    user_embedding = Reshape((d,))(user_embedding)
    
    item = Input(name = 'item', shape = [None,])
    item_embedding = Embedding(name = 'item_embedding',
                               input_dim = num_items,
                               output_dim = d, input_length = 1,
                            embeddings_constraint=non_neg())(item)
    Dropout(0.2)
    item_embedding = BatchNormalizetion()(item_embedding)
    item_embedding = Reshape((d,))(item_embedding)
    
    merged = Dot(name = 'dot_product',axes = 2)([user_embedding, item_embedding])

    merged = Reshape(target_shape = [1])(merged)
    
    out = Dense(1, activation = 'sigmoid')(merged)
    model = Model(inputs = [user, item], outputs = out)
    
    model.compile(optimizer = 'sgd', loss = 'binary_crossentropy', 
                  metrics = ['accuracy'])
    return model

In [157]:
model = embedding_model(d = 128)

In [166]:
def generate_batch(pairs, n_positive = 50, negative_ratio = 1.0):
    batch_size = n_positive * (1 + negative_ratio)
    batch = np.zeros((batch_size, 3))
    
    while True:
        # Randomly choose positive examples
        for idx, (uid, item) in enumerate(random.sample(pairs, n_positive)):
            batch[idx, :] = (uid, item, 1)
        idx += 1
        
        while idx < batch_size:
            random_user = random.randrange(num_users)
            random_item = random.randrange(num_items)
            # Check 
            if (random_user, random_item) not in pairs:
                batch[idx, :] = (random_user, random_item, 0)
                idx += 1
                
        # Make sure to shuffle order
        np.random.shuffle(batch)
        yield {'user': batch[:, 0], 'item': batch[:, 1]}, batch[:, 2]
        
        
n_positive = 50
gen = generate_batch(pairs, n_positive, negative_ratio = 1)

In [168]:
h = model.fit_generator(gen, epochs = 100, steps_per_epoch = 20)

Epoch 1/100
20/20 [==============================] - 6s 313ms/step - loss: 0.6577 - acc: 0.7460
Epoch 2/100
20/20 [==============================] - 6s 308ms/step - loss: 0.6550 - acc: 0.7385
Epoch 3/100
20/20 [==============================] - 6s 313ms/step - loss: 0.6491 - acc: 0.7480
Epoch 4/100
20/20 [==============================] - 6s 308ms/step - loss: 0.6485 - acc: 0.7375
Epoch 5/100
20/20 [==============================] - 6s 306ms/step - loss: 0.6436 - acc: 0.7430
Epoch 6/100
20/20 [==============================] - 6s 308ms/step - loss: 0.6423 - acc: 0.7330
Epoch 7/100
20/20 [==============================] - 6s 308ms/step - loss: 0.6353 - acc: 0.7475
Epoch 8/100
20/20 [==============================] - 6s 306ms/step - loss: 0.6319 - acc: 0.7480
Epoch 9/100
20/20 [==============================] - 6s 306ms/step - loss: 0.6277 - acc: 0.7500
Epoch 10/100
20/20 [==============================] - 6s 305ms/step - loss: 0.6251 - acc: 0.7485
Epoch 11/100
20/20 [===================

In [185]:
# Extract embeddings
user_layer = model.get_layer('user_embedding')
user_weights = user_layer.get_weights()[0]

item_layer = model.get_layer('item_embedding')
item_weights = item_layer.get_weights()[0]

In [186]:
item_weights.transpose().shape

(64, 3260)

In [187]:
interaction = np.dot(user_weights , (item_weights.transpose()))

In [188]:
interaction

array([[ 0.01120719, -0.00746568, -0.0018558 , ...,  0.01073264,
        -0.00122674, -0.00085257],
       [-0.01111018, -0.00301666,  0.01529985, ..., -0.01552236,
         0.00479521, -0.00679149],
       [ 0.003576  , -0.01293721, -0.00504763, ...,  0.01381103,
        -0.00120778,  0.00075167],
       ...,
       [ 0.00670452,  0.02049342,  0.0004959 , ..., -0.01606832,
         0.00843401, -0.00921321],
       [ 0.00842183, -0.01000228, -0.00532983, ...,  0.0002222 ,
        -0.00259108, -0.01552451],
       [ 0.01696286, -0.00850848, -0.00814447, ..., -0.01213065,
        -0.01251527,  0.01836075]], dtype=float32)

In [189]:
predict = pd.DataFrame(columns=['UserId','ItemId'])

In [190]:
for uid, items in enumerate(interaction):
    for i in user_item.iloc[uid][1]:
        items[i] = -99
    topk = np.argsort(-items)[:50]
    predict.loc[uid,'UserId'] = uid
    predict.loc[uid,'ItemId'] = ' '.join([str(x) for x in topk])

In [191]:
predict.to_csv('submit.csv',index = 0)

In [192]:
predict.shape

(4454, 2)